In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve
import cv2
import pickle

In [2]:
def init_tumor2(size, radius):
    """
     initializes the field phi with a circle
     :param size: size of matrix
     :param radius: tumor initial radius (created in center)
     :return param phi: order parameter matrix
    """
    cx,cy = int(size/2),int(size/2)
    x, y = np.indices((size, size))
    phi = (np.abs(np.hypot(x-cx, y-cy)) < radius).astype(float)
    return phi

In [3]:
def init_tumor_ellipse(size,a,b,radius):
    cx,cy = int(size/2),int(size/2)
    x, y = np.indices((size, size))
    phi = (np.abs(np.hypot((x-cx)/a, (y-cy)/b)) < radius).astype(float)
    return phi

In [4]:
def init_s2(size):
    s = 2.55 + 0.4 * np.random.rand(size,size)
    return s

In [5]:
def fintegrate_nophi(phi, sigma, S, dt, h, **kwargs):
    """
      try to go even faster
      -> don't create copy here
      -> try to reduce multiplications
      -> cv2 for convolution?
      integrate the order parameter phi and the chemical field sigma one tstep
      with 5x5 stencil (five point stencil - https://en.wikipedia.org/wiki/Five-point_stencil)
    :param phi: order parameter
    :param sigma: nutrient
    :param S: source of nutrient
    :param tstep: total number of timesteps
    :param dt: time increment
    :param kwargs: dictionary of parameters lambda, tau, chi, A, epsilon, delta, gamma
    :return:
    """
    stencil_5 = (1.0 / (12.0 * h * h)) * np.array(
        [[0, 0, -1, 0, 0],
         [0, 0, 16, 0, 0],
         [-1, 16, -60, 16, -1],
         [0, 0, 16, 0, 0],
         [0, 0, -1, 0, 0]])

    phinew = phi + dt * (kwargs['lambda_'] * cv2.filter2D(phi,-1,stencil_5,borderType=cv2.BORDER_REPLICATE) +
                      (64.0/kwargs['tau']) * phi * (1.0 - phi) * (phi - 0.5) +
                      kwargs['chi'] * sigma - kwargs['A'] * phi)

    sigmanew = sigma + dt * (kwargs['epsilon']*kwargs['lambda_'] * cv2.filter2D(sigma,-1,stencil_5,borderType=cv2.BORDER_REPLICATE) +
                              S - kwargs['delta'] * phi - kwargs['gamma'] * sigma)    

    return phinew, sigmanew

In [6]:
def run_partest(phistart, sigmastart, Sstart, param):
    print("running parameter test with parameters", param)
    phiold = np.copy(phistart)
    sigmaold = np.copy(sigmastart)
    S = np.copy(Sstart)
    phiarr = np.zeros((param['days'],size,size)) #take care, size not derived from input!
    sigmaarr = np.zeros((param['days'],size,size))
    
    tstep = param['days']/param['dt']
    
    for iteration in np.arange(tstep):
        phinew, sigmanew = fintegrate_nophi(phiold, sigmaold, S, **param)
        phiold = phinew
        sigmaold = sigmanew

        if iteration%int(1/param['dt']) == 0:
            phiarr[int(iteration*param['dt'])]=phinew
            sigmaarr[int(iteration*param['dt'])]=sigmanew
            print("day",iteration*param['dt'])
            #plt.imshow(phiold,vmin=0,vmax=1)
            #plt.imshow(sigmaold,vmin=0,vmax=3)
            #plt.show()    
    results = {'param':param,'phistart':phistart,'sigmastart':sigmastart,'Sstart':Sstart,'phiarr':phiarr,'sigmaarr':sigmaarr}
    return results

In [7]:
Avals = list(np.arange(0.3,2.0,0.1))
chivals = [1.09]
#deltavals = list(np.arange(2.2,3.2,0.1))

In [8]:
size = 200
phistart = init_tumor_ellipse(size,1,1.3,50)
sigmastart = np.ones((size,size),dtype=float)
Sstart = init_s2(size)
param = {'dt':0.00025, 'h':5., 'lambda_':432., 'epsilon':40., 
         'A':1.2, 'gamma':2.74, 'tau':3.65, 'chi':1.09, 'delta':2.75, 'days':2}

for A in Avals:
    for chi in chivals:
        param['A'] = A
        param['chi'] = chi
        res = run_partest(phistart, sigmastart, Sstart, param)
        
        filename = 'parvariation/A_'+str(A)+'_chi_'+str(chi)+'.pickle'
        with open(filename, 'wb') as handle:
            pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

running parameter test with parameters {'gamma': 2.74, 'chi': 1.09, 'h': 5.0, 'tau': 3.65, 'delta': 2.2000000000000002, 'lambda_': 432.0, 'dt': 0.00025, 'epsilon': 40.0, 'days': 2, 'A': 0.29999999999999999}
day 0.0
day 1.0
running parameter test with parameters {'gamma': 2.74, 'chi': 1.09, 'h': 5.0, 'tau': 3.65, 'delta': 2.3000000000000003, 'lambda_': 432.0, 'dt': 0.00025, 'epsilon': 40.0, 'days': 2, 'A': 0.29999999999999999}
day 0.0
day 1.0
running parameter test with parameters {'gamma': 2.74, 'chi': 1.09, 'h': 5.0, 'tau': 3.65, 'delta': 2.4000000000000004, 'lambda_': 432.0, 'dt': 0.00025, 'epsilon': 40.0, 'days': 2, 'A': 0.29999999999999999}
day 0.0
day 1.0
running parameter test with parameters {'gamma': 2.74, 'chi': 1.09, 'h': 5.0, 'tau': 3.65, 'delta': 2.5000000000000004, 'lambda_': 432.0, 'dt': 0.00025, 'epsilon': 40.0, 'days': 2, 'A': 0.29999999999999999}
day 0.0
day 1.0
running parameter test with parameters {'gamma': 2.74, 'chi': 1.09, 'h': 5.0, 'tau': 3.65, 'delta': 2.600000

KeyboardInterrupt: 